In [ ]:
import os
import re
from datetime import datetime

DATE_FORMAT = "%Y-%m"
DATE_REGEXP = r"\d{4}-\d{2}"


def in_interval(date_interval: list, string_with_date: str,
                date_format: str, regexp: str) -> bool:
    """
    Checks is date which included in string_with_date in date interval
    :param date_interval: dates interval [start_date, end_date]
    :param string_with_date: string which include date in some format
    :param date_format: date format constant for method datetime.strptime()
    :param regexp: regexp filter for finding date in string_with_date
    :return: True or False
    """
    if re.search(regexp, re.sub(r"(\\+|/+)", '-', string_with_date)):
        return date_interval[0] <= datetime.strptime(
            re.search(
                regexp,
                re.sub(r"(\\+|/+)", '-', string_with_date)
            ).group(), date_format
        ) <= date_interval[1]
    else:
        return False


def list_directory(dir_path: str, date_interval: list) -> list:
    """
    Lists directory (dir_path) and returns all files which in date_interval
    :param dir_path: path to directory with target files
    :param date_interval: dates interval [start_date, end_date]
    :return: list of file names
    """
    return [
        "{}/{}".format(dir_path, file_name)
        for file_name in os.listdir(dir_path)
        if os.path.isfile(
            os.path.join(dir_path, file_name)
        ) and in_interval(date_interval, file_name, DATE_FORMAT, DATE_REGEXP)
    ]


def walk_directories(dir_path: str, date_interval: list) -> list:
    """
    Walks directory (dir_path) and returns all files which path in date
    interval
    :param dir_path: path to directory with target files
    :param date_interval: dates interval [start_date, end_date]
    :return: list of file names
    """
    return [
        "{}/{}".format(directory_info[0], directory)
        for directory_info in os.walk(dir_path)
        for directory in directory_info[-1]
        if in_interval(
            date_interval, directory_info[0], DATE_FORMAT, DATE_REGEXP
        ) and os.path.isfile(directory_info[0] + '/' + directory)
    ]


def find_files(dir_path: str, start_date: datetime,
               end_date=None, method='listdir') -> list:
    """
    Returns all files from target directory which in dates interval
    :param dir_path: path to directory with target files
    :param start_date: start date in interval
    :param end_date: end date in interval
    :param method: search method
    :return: list of file names
    """
    assert isinstance(start_date, datetime), "start_date must be datetime"
    assert end_date is None or isinstance(end_date, datetime), \
        "end_date must be datetime"
    date_interval = [start_date, end_date] \
        if end_date is not None else [start_date, start_date]
    if method == 'listdir':
        return list_directory(dir_path, date_interval)
    if method == 'walkdir':
        return walk_directories(dir_path, date_interval)


def load_in_memory(files: list, encoding='utf-8') -> list:
    """
    Load files to memory
    :param files: list of file names
    :param encoding: files encoding for method open()
    :return: list with files content
    """
    return [open(file, 'r', encoding=encoding).read() for file in files]

In [ ]:
DIR_PATH = "./test_set"
files = find_files(DIR_PATH, datetime(2014, 5, 1), method='listdir')
print(files)
payload = load_in_memory(files, 'koi8-r')
print(payload)